In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 100,
    separators= ['\n\n', '\n']
)

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(
    "./data/income_tax.md",
    mode="single",
    strategy="fast",
)
docs = loader.load_and_split(text_splitter)
docs

In [ ]:
# md to txt for table
import markdown
from bs4 import BeautifulSoup

with open('./data/income_tax.md', 'r', encoding='utf-8') as md_file:
    md_content = md_file.read()

html_content = markdown.markdown(md_content)
soup = BeautifulSoup(html_content, 'html.parser')
text_content = soup.get_text()

with open('./data/income_tax.txt', 'w', encoding= 'utf-8') as txt_file:
    txt_file.write(text_content)


In [9]:
from langchain_community.document_loaders import TextLoader
txt_loader = TextLoader(
    './data/income_tax.txt',
    encoding = 'utf-8'
)
txt_docs = txt_loader.load_and_split(text_splitter)
txt_docs[:5]

[Document(metadata={'source': './data/income_tax.txt'}, page_content='소득세법\n[시행 2025. 10. 1.] [법률 제21655호, 2025. 10. 1., 타법개정]\n기획재정부 (세제실세제과) 044-215-4312\n기획재정부 (소득세제과) 044-215-4216\n기획재정부 (금융세제과) 044-215-4233\n기획재정부 (소득세제과(사업소득, 기타소득)) 044-215-4217  \n제1장 총칙\n제조(목적)\n이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 2009. 12. 31.]\n제2조(정의)\n이 법에서 사용하는 용어의 뜻은 다음과 같다. [개정 2010. 12. 27, 2014. 12. 23, 2018. 12. 31.]\n\n“거주자”란 국내에 주소를 두거나 183일 이상 거소(居所)를 둔 개인을 말한다.\n“비거주자”란 거주자가 아닌 개인을 말한다.\n“내국법인”이란 법인세법 제2조제1호에 따른 내국법인을 말한다.\n“외국법인”이란 법인세법 제2조제3호에 따른 외국법인을 말한다.\n“사업자”란 사업소득이 있는 거주자를 말한다.\n제5항에 따른 사업자·소득자·비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무)\n구분리스트.\n\n거주자\n비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n거주자\n비거주자\n내국법인\n외국법인및 국내사업장 또는 국내영업소(출처소, 그 밖에 이에 준하는 것을 포함한다. 이하 같다)\n그 밖에 법에서 정하는 원천징수의무자\n“국세기본법” 제23조제1항에 따른 법인이 아닌 단체 같은 제2항에 따른 법인이 보는 단체로서 “법인은 법인 외의 

In [11]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [12]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    txt_docs,
    embeddings,
    collection_name='income_tax_collection',
    persist_directory='./data/income_tax_collection'
)

In [17]:
retriever = vector_store.as_retriever()

In [18]:
query = '피고용보험 가입자의 소득세는?'
retriever.invoke(query)

[Document(id='8dd27fc1-9e2b-4f48-a54d-184b9ef7f003', metadata={'source': './data/income_tax.txt'}, page_content='위와 같은 경우에 제세액과 별도로 평균화 구간 명칭, 전환형의 100분의 10을 300만원(전환 세액기준에 해당 납입한 경우에는 300만원에 기초 납입액이 적용되는 금액을 차감한 금액으로 한다) 그 목적 금액과 제척 장 조세를 통해 연금계좌의 납입한 금액으로 하는 금액을 제정부서 제58행정이 규정에 따라 연금계좌세액정산 및 신청 적절 등 관리에 필요한 사항은 대동 범위로 한다. <개정 2019. 12. 31.>\n[분조신설 2014. 1. 1.]\n제59조(특별세액공제) ① 근로소득이 있는 거주자(일용근로자는 제외한다. 이하 이 조에서 같다)가 해당 과세기간에 만기와 환급되는 금액에 납입보험료를 초과하지 아니하는 보험의 보험혜택에 따라 지급받는 다음 각 호의 보험료를 지출세액에서 공제한다. 다만, 다음 각 호의 보험료로 각 과세기간에 각각 100만 원을 초과하는 경우 그 초과하는 금액은 각락 있는 것으로 한다. <개정 2015. 5. 13.>'),
 Document(id='8af133e6-67c1-4cdf-a775-b5bdd6a69cda', metadata={'source': './data/income_tax.txt'}, page_content='기초공제대상자 중 장애인인 피부양자 또는 수급자 하는 장애인전용보험으로서 지급하는 보험료\n기초공제대상자를 피부법자로 하는 대출형량으로 정하는 보험료(제1호에 따른 장애인전용장애보험으로 제정한다)\n\n② 근로소득이 있는 거주자가 기본공제대상자(나이 및 소득의 제한을 받지 아니한 다를) 위하여 해당 과세기간에 대 통합적으로 정하는 의료비를 지급한 경우 다음 각 호의 금액의 15/제3호의 경우에는 100분의 20, 제4호의 경우에는 100분의 30에 해당하는 금액을 해당 과세기간의 종합소득세에서 공제한다. <개정 2014

In [20]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str


In [21]:
from langgraph.graph import StateGraph
graph_builder = StateGraph(AgentState)

In [22]:
def retrieve(state: AgentState):
    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [55]:
#같이 담아줄 프롬프트 랭스미스
from langsmith import Client
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")


In [29]:
def generate(state: AgentState):
    query = state['query']
    context = state['context']
    rag_chain = prompt|llm
    response = rag_chain.invoke({'question':query, 'context':context})
    return {'answer':response}

In [ ]:
from langgraph.graph import START, END

graph_builder.add_node('retrieve',retrieve)
graph_builder.add_node('generate',generate)


graph_builder.add_edge(START, 'retrieve')
graph_builder.add_edge('retrieve', 'generate')
graph_builder.add_edge('generate', END)

In [34]:
graph = graph_builder.compile()

In [48]:
sequence_graph_builder = StateGraph(AgentState).add_sequence([retrieve,generate])

In [50]:
sequence_graph_builder.add_edge(START, 'retrieve')
sequence_graph_builder.add_edge('generate', END)

In [57]:
query = '소득세법에서 정의하는 종합소득의 종류는 무엇인가?'
initial_state = {'query': query}
graph.invoke(initial_state)

{'query': '소득세법에서 정의하는 종합소득의 종류는 무엇인가?',
 'context': [Document(id='247ccd0d-0f9f-4f49-9050-a20ee5811a00', metadata={'source': './data/income_tax.txt'}, page_content='종합소득에 대한 과세표준(이하 “종합소득과세표준”이라 한다)은 제16조, 제17조, 제19조, 제20조, 제21조, 제22조부터 제26조까지, \n제27조부터 제29조까지, 제31조부터 제35조까지, 제37조, 제39조, 제41조부터 제46조까지, 제46조의2, 제47조에 따라 제47조의2에 따른 이자소득금, 배당소득금, 사업소득금, 근로소득금, 액면금소득 및 기타소득의 합계액(이하 “종합소득금액”이라 한다)은 다음의 비율로 제50조, 제51조, 제51조의2에 따른 총량에 따라 “종합소득공제”를 적용하고 한다. <개정 2013. 1. 1. , 2014. 1. 1.>\n당 정각 혀에 따른 소득의 종합소득과세표준을 계산할 때 한산하지 아니다.<개정 2010. 12. 27., 2011. 7. 14., 2013. 1. 1., 2014. 12. 23., 2015. 12. 15., 2017. 12. 19., 2018. 12. 1., 2019. 12. 31., 2020. 12. 29. , 2023. 12. 1.>'),
  Document(id='731ac11c-5549-4caa-b2c1-0bf68c1ef0c7', metadata={'source': './data/income_tax.txt'}, page_content='제64조의3(분리과세기타소득에 대한 세액 계산의 특례)\n제14조에 따라 종합소득과세표준을 계산할 때 제127조제1항제 1항제6호나목의 소득을 합산하지 아니하는 경우 그 합산하지 아니하는 기타소득에 대한 계산세액은 해당 기타소득 금액에 제120조제1항제6호의 세율을 적용하여 계산한 금액으로 한다.<개정 2020. 12. 29.> \n1) 제180조의4의4의 소득을 합